# RijksMuseum

https://data.rijksmuseum.nl/object-metadata/api/

    `To start using the data and images, you first need to obtain an API key by registering for a Rijksstudio account. You will be given a key instantly upon request, which you can find at the advanced settings of your Rijksstudio account.`

In [7]:
import requests

import json

import os
from IPython.core.display import display, HTML

from cromulent.model import factory
    
from lib import linkedart as la



def fn(dir):       # 1.Get file names from directory
    file_list=os.listdir(dir)
    for file in file_list:
        display(HTML("<a target='_new' href='" + dir + file +"'>" + file + "</a>"))
    

In [8]:
keyfile = "data/ruskin/input/keyrijks.txt"
key = open(keyfile, mode='r', encoding='utf-8-sig').read()
url = "https://www.rijksmuseum.nl/api/nl/collection?key=" + key + "&involvedMaker=John+Ruskin"

rijksdir = "data/rijks/json/"

rijksvis = "data/ruskin/output/json/"

baseURI = "http://www.rijksmuseum.nl/nl/collectie/"



In [9]:
response = requests.get(url)
data = response.json()

for artwork in data["artObjects"]:
    print(json.dumps(artwork,indent=2))
    
    if artwork["hasImage"] == False:
        continue
    if artwork["principalOrFirstMaker"] != "John Ruskin":
        continue
    text_file = open( rijksdir + artwork["id"] + ".json", "wt")
    text_file.write(json.dumps(artwork, indent=2))
    text_file.close()

{
  "links": {
    "self": "http://www.rijksmuseum.nl/api/nl/collection/RP-P-1942-556",
    "web": "http://www.rijksmuseum.nl/nl/collectie/RP-P-1942-556"
  },
  "id": "nl-RP-P-1942-556",
  "objectNumber": "RP-P-1942-556",
  "title": "Kustlijn met kliffen",
  "hasImage": false,
  "principalOrFirstMaker": "John Ruskin",
  "longTitle": "Kustlijn met kliffen, John Ruskin, 1855",
  "showImage": false,
  "permitDownload": true,
  "webImage": null,
  "headerImage": {
    "guid": null,
    "offsetPercentageX": 0,
    "offsetPercentageY": 0,
    "width": 0,
    "height": 0,
    "url": null
  },
  "productionPlaces": [
    "New York"
  ]
}
{
  "links": {
    "self": "http://www.rijksmuseum.nl/api/nl/collection/RP-F-2001-7-481",
    "web": "http://www.rijksmuseum.nl/nl/collectie/RP-F-2001-7-481"
  },
  "id": "nl-RP-F-2001-7-481",
  "objectNumber": "RP-F-2001-7-481",
  "title": "Roadside songs of Tuscany",
  "hasImage": true,
  "principalOrFirstMaker": "George Allen",
  "longTitle": "Roadside song

In [10]:
mapp = {
    
    "id":"id",
    "accession_number":"objectNumber",
    "classification" : "",
    "title": "title",
    "alt_title": "longTitle",
    "notes": "",
    "date_created":"",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"principalOrFirstMaker",
    "physical_medium": "",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "",
    "created_provenance": "" ,
    "credit_line": "",
    "collection" : "",
    "classification": "",
    "current_status" : "",
    "current_owner": "",
    "image_url":"",
    "homepage" : ""
}

In [15]:
import re

def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
               
                if prop == "alt_title":
                    alt_title = obj[key]
                    # parse alt_title for dates
                    
                    years = re.findall('(\d{4})', alt_title)
                    if years[0]:
                        objProp["date_created_earliest"] = years[0]
                    if years[1]:
                        objProp["date_created_latest"] = years[1]
                        objProp["date_created"] = years[0] + "-" + years[1]
                    else:
                        objProp["date_created"] = years[0]

                else:
                    objProp[prop] = obj[key]
                    
    objProp["current_owner"] = {"name":"Rijksmuseum",
                                "location":"Amsterdam, Netherlands",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": "Museum"}
    return objProp   


file_list=os.listdir(rijksdir)
for file in file_list:
    with open(rijksdir + file) as json_file:
        obj = json.load(json_file)
        objProp=createObjProp(obj,mapp)
        
        objProp["image_url"] = obj["webImage"]["url"]
        objProp["creator"] = [{"id": baseURI + "creator/" + obj["id"], "name": obj["principalOrFirstMaker"], "role": "Artist"}]
        
        objProp["classification"] = "Painting"
        objProp["homepage"] = obj["links"]["web"]
        
        
        
        if objProp["creator"] != "":
            print(objProp)
            
            id = str(objProp["id"])
            object_uri = baseURI + id
    
            # create obj description
            objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
               
            # write to file 
            text_file = open(rijksvis + id + ".json", "wt")
            n = text_file.write(factory.toString(objLA, compact=False))
            text_file.close()
            

{'id': 'nl-RP-T-1987-19', 'accession_number': 'RP-T-1987-19', 'title': 'Bergkam aan het Meer van Genève', 'creator': [{'id': 'http://www.rijksmuseum.nl/nl/collectie/creator/nl-RP-T-1987-19', 'name': 'John Ruskin', 'role': 'Artist'}], 'date_created_earliest': '1829', 'date_created_latest': '1900', 'date_created': '1829-1900', 'current_owner': {'name': 'Rijksmuseum', 'location': 'Amsterdam, Netherlands', 'type': 'http://vocab.getty.edu/aat/300312281', 'type_label': 'Museum'}, 'image_url': 'https://lh3.ggpht.com/SOPNNYPF1S5Ij-DKCcm7AoweEHz--rJ0VPCh5WZdFbCKEPNnSI9ZAzYQz_nxKVtuurYAnOEvSFfYv5ZNcX-Ma7TwNH0=s0', 'classification': 'Painting', 'homepage': 'http://www.rijksmuseum.nl/nl/collectie/RP-T-1987-19'}
{'id': 'nl-RP-T-1957-335', 'accession_number': 'RP-T-1957-335', 'title': 'Gezicht op S. Anastasia te Verona, over de Adige', 'creator': [{'id': 'http://www.rijksmuseum.nl/nl/collectie/creator/nl-RP-T-1957-335', 'name': 'John Ruskin', 'role': 'Artist'}], 'date_created_earliest': '1829', 'dat

In [16]:
fn(rijksvis)